### Watershed Indicators Setup

The US EPA publishes an important set of tracking indicators for every watershed in the continental US through a program called Environmental Atlas.

There are 240 indicators in this set, and approxiately 8,000 watersheds (at "HUC12 level")

While it is possible to explore this dataset using the EnviroAtlas online web mapping tool, that system only lets you explore a single variable at a time.  With 240 variables, that can be rather cumbersome.

Putting data like this in MapD supports a different form of exploratory analysis, where all indicators are simultaneously and interactively available.  This allows you to create high-level syntheses of the information, as well as to explore the relationships between indicators.

In [ ]:
import pandas as pd
import os
from pymapd import connect

In [ ]:
#parameters

# put stuff here
base_path = '/home/mapdadmin/demo/wshed'

# URLs for regional xcel files are located
base_indicators_url = 'https://www.epa.gov/sites/production/files/2017-02/170209wsio_indicator_data_v2.0_epa_region{}.xlsx'

# the format of the downloaded excel file names
indicators_base = '170209wsio_indicator_data_v2.0_epa_region{}.xlsx'

# name for our regional csv intermediate files
csv_base_filename = 'wsio_indicators_region{}.csv'

# output table
table_name = 'Watershed_Indicators' #name of derised output table in mapd

Get the data (sequentially, since only a few, but could also use xargs to get in parallel)

In [ ]:
import os
for region in range(1,10):
    zero_padded_region = '{:02d}'.format(region)
    specific_url = base_indicators_url.format(zero_padded_region)
    output_xls = indicators_base.format(zero_padded_region)
    if not os.path.exists(output_xls):
        print("Getting {}".format(specific_url))
        !wget {specific_url}

Because our data are burried within a specific Excel sheet, we need to read each spreadsheet into memory, grabbing only the correct sheet, and then save back out to CSV.  We could for speed upload directly to MapD from a dataframe, using pymapd.  However in this case we prefer to keep an explicit copy on disk for public sharing in an open format.

In [ ]:
import csv
for region in range(1,2):
    zero_padded_region = '{:02d}'.format(region)
    specific_file = indicators_base.format(zero_padded_region)
    print("Processing {}".format(specific_file))

    of = csv_base_filename.format(zero_padded_region)
    df = pd.read_excel(specific_file, sheet_name='WSIO_REGIONAL_DATA_TABLE')


In [ ]:
df.head()

In [ ]:
orig_col_names = df.columns
sql_legal_col_names = []
for cn in orig_col_names:
    print(cn)
    sql_legal_col_name = cn.title().replace(' ','').replace('%','Pcnt')
    sql_legal_col_name = "".join(ch for ch in sql_legal_col_name if ch.isalnum())
    #cn.title().replace(' ','').replace('(','').replace(')','').replace('%','Pcnt').replace(',','-')
    print("   {}".format(sql_legal_col_name))
    sql_legal_col_names.append(sql_legal_col_name)

In [ ]:
name_offset = 0
for cn in orig_col_names:
    newname = sql_legal_col_names[name_offset]
    print('Renaming col \"{}\" to \"{}\"'.format(cn, newname))
    name_offset += 1
    
    

In [ ]:
import csv
for region in range(1,10):
    zero_padded_region = '{:02d}'.format(region)
    specific_file = indicators_base.format(zero_padded_region)
    print("Processing {}".format(specific_file))

    of = csv_base_filename.format(zero_padded_region)
    if not os.path.exists(of):
        df = pd.read_excel(specific_file, sheet_name='WSIO_REGIONAL_DATA_TABLE')
        df.columns = sql_legal_col_names  # force our new column names

        # later, after debugging, add compression=gzip
        df.to_csv(of, index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
!ls *.csv

To Do: Figure out method for prettier column name truncation meeting MapD specs.  Current EPA column names are very long, and include commas.  MapD robustly converts and truncates them - but results can be hard to interpret.

In [ ]:
# connect to mapd default database
con = connect(user="mapd", password= "HyperInteractive", host='localhost', dbname="mapd")
con

In [ ]:
with open('wsio_indicators_create.sql', 'r') as myfile:
    wsio_indicators_create=myfile.read().replace('\n', '')

In [ ]:
try:
    con.execute(wsio_indicators_create)
except:
    print('Problem creating database')

In [ ]:
for region in range(1,10):
    zero_padded_region = '{:02d}'.format(region)
    csv_file = csv_base_filename.format(zero_padded_region)
    append_table_command = "COPY {} FROM '{}'".format(table_name, os.path.join(base_path,csv_file))
    try:
        print('Executing MapD command: {}'.format(append_table_command))
        con.execute(append_table_command)
    except:
        print('Problem appending to master indicators table')

In [ ]:
!head /home/mapdadmin/demo/wshed/wsio_indicators_region01.csv

In [ ]:
!tail /var/lib/mapd/data/mapd_log/mapd_server.ERROR

### Get Watershed Boundary Files

In [ ]:
wshed_shape = 'ftp://newftp.epa.gov/epadatacommons/ORD/EnviroAtlas/NHDPlusV2_WBDSnapshot_EnviroAtlas_CONUS.gdb.zip'

In [ ]:
!wget {wshed_shape}

In [ ]:
wshed_gdb_zip = 'NHDPlusV2_WBDSnapshot_EnviroAtlas_CONUS.gdb.zip'
gdal_input = '/vsizip/{}'.format(wshed_gdb_zip) 
import sys,os,os.path
os.environ['CPL_ZIP_ENCODING']='UTF-8'
!gdalwarp -t_srs 'EPSG:4326' {gdal_input} 'us_watersheds_huc12.shp'

In [ ]:
import geopandas as gpd

wshed = gpd.read_file(wshed_shape)

In [ ]:
wshed.head()

In [ ]:
!unzip {wshed_gdb}

In [ ]:
import sys,os,os.path
os.environ['CPL_ZIP_ENCODING']='UTF-8'
wshed_gdb = 'NHDPlusV2_WBDSnapshot_EnviroAtlas_CONUS.gdb'
# -csql statement  
# HUC_12: String (12.0)
sql = 'SELECT *, CAST(HUC_12 AS INTEGER) FROM NHDPlusV2_WBDSnapshot_EnviroAtlas_CONUS'
command = ogr2ogr -t_srs 'EPSG:4326' -sql {sql} {wshed_gdb} 'us_watersheds_huc12.shp'
!ogr2ogr -t_srs 'EPSG:4326' -csql {sql} {wshed_gdb} 'us_watersheds_huc12.shp'

In [ ]:
!ogr2ogr --help

In [36]:
!ogr2ogr -t_srs 'EPSG:4326' -sql 'SELECT Pct_Land, CAST(HUC_12 AS INTEGER) FROM NHDPlusV2_WBDSnapshot_EnviroAtlas_CONUS' 'us_watersheds_huc12.shp' NHDPlusV2_WBDSnapshot_EnviroAtlas_CONUS.gdb 

In [ ]:
# should probably drop Shape_Area column because its overflowing

In [ ]:
select us_watersheds_huc12.*,watershed_indicators_epa_region3.* from us_watersheds_huc12 INNER JOIN watershed_indicators_epa_region3 ON us_watersheds_huc12.HUC_122 = watershed_indicators_epa_region3.HydrologicUnitCode12DigitHUC12 ; 